This notebook contains the code to evaluate within subject cross session gradient peak locations. 
It also is set up to create ROIs for distance measurements to the rest of the cortex 

In [1]:
import numpy as np 
import nibabel as nib 

import nilearn.plotting
import matplotlib.pyplot as plt
import mayavi
from mayavi import mlab
import ptitprince as pt
import seaborn as sn
import pandas as pd

from utils import *

import gdist
import surfdist as sd 
import surfdist.analysis  

from sklearn.cluster import KMeans

from IPython.display import display, HTML
import matplotlib.offsetbox as offsetbox
!jupyter nbextension enable mayavi --user --py
mlab.init_notebook('png',1500,1500)


/Users/austin/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


Enabling notebook extension mayavi/x3d/x3dom...
      - Validating: OK
Notebook initialized with png backend.


In [2]:
subjects=!ls -d *10*/
for i in range(len(subjects)):
    subjects[i]=subjects[i].split('/')[0]
#### load watersheds
LWS=nib.load('/Users/austin/Documents/ParisHorizontal/Mai2022Grads/watershedTemplates/LWS.28.max.label.gii').darrays[0].data
RWS=nib.load('/Users/austin/Documents/ParisHorizontal/Mai2022Grads/watershedTemplates/RWS.28.max.label.gii').darrays[0].data

By defining the hcp_subj class, operations are relatively contained

In [3]:
import numpy as np 
import nibabel as nib 

import nilearn.plotting
import matplotlib.pyplot as plt
import mayavi
from mayavi import mlab
import ptitprince as pt
import seaborn as sn
import pandas as pd

from utils import *

import gdist
import surfdist as sd 
import surfdist.analysis  

from sklearn.cluster import KMeans

def recort(X,fill,dims):
    out=np.zeros(dims)
    out[fill]=X
    return out

def recort_bin(X,fill,dims):
    out=np.zeros(dims)
    mini=np.zeros(len(fill))
    mini[X]=1
    out[fill]=mini
    return out
    

def binit(X):
    pct=np.percentile(X,[10,20,30,40,50,60,70,80,90])
    pct=np.digitize(X,pct)
    return pct+1 

def binit20(X):
    pct=np.percentile(X,[20,40,60,80])
    pct=np.digitize(X,pct)
    return pct+1 


def get_zoneVerts(WS):
    zoneverts={}
    for i in range(1,np.max(WS)):
        zoneverts.update({f'zone{i}':np.where(WS==i)[0]})
    return zoneverts

def oh_mayavi(surf,stat,cmap,clrbar=True):
    """surface, statmap, colormap"""
    ##### parse the gifti 
    anat=nib.load(surf)
    coords=anat.darrays[0].data
    x=coords[:,0]
    y=coords[:,1]
    z=coords[:,2]
    triangles=anat.darrays[1].data
    ##### if subcortical mask provided use it

#         print('masking out subcortex')
#     sub_cort=nilearn.surface.load_surf_data(args[0])
#     stat[sub_cort]=float('NaN')

    
    
    ### start mayavi 
    
    maya=mlab.triangular_mesh(x,y,z,triangles,scalars=stat,colormap=cmap)
    
    mlab.view(azimuth=0, elevation=-90)
    f = mlab.gcf()
    cam = f.scene.camera
    cam.zoom(1.)
    if clrbar==True:
        cb=mlab.colorbar(orientation='vertical', nb_labels=3,label_fmt='%.2f')
        cb.label_text_property.color =(0,0,0)
    else: 
        pass

    mlab.draw()
    

    img1=mlab.screenshot(figure=maya,mode='rgba',antialiased=True)
    mlab.view(azimuth=0, elevation=90)
    mlab.figure(bgcolor=(0, 0, 0))
    ### clear figure
    mayavi.mlab.clf()
    
    f = mlab.gcf()
    cam = f.scene.camera
    cam.zoom(1.1)
    mlab.draw()
    img2=mlab.screenshot(figure=maya,mode='rgba',antialiased=True)
    ### clear figure
    mayavi.mlab.clf()
    mlab.clf()
    mlab.close()
    return img1,img2


def plot_srfs(a,b,c,d):
    figure=plt.figure(figsize=(6, 8), dpi=180)
    plt.subplot(2,2,1)
    plt.imshow(a)
    plt.axis('off')
    plt.subplot(2,2,2)
    plt.imshow(b)
    plt.axis('off')
    mlab.clf()
    plt.subplot(2,2,3)
    plt.imshow(c)
    plt.axis('off')
    plt.subplot(2,2,4)
    plt.imshow(d)
    plt.axis('off')
    

    plt.subplots_adjust(left=0.1,
                        bottom=0.5, 
                        top=0.9, 
                        wspace=0, 
                        hspace=0)
#     plt.savefig(f'{file}.png',bbox_inches='tight',facecolor='w')
#     plt.close()


def dice_it(A,B):
    
    num=2*(len(np.intersect1d(A,B)))
    den=len(A)+len(B)
    
    if den ==0:
        return np.nan
    else:
        return num/den

def jaccard_it(A,B):
    num=len(np.intersect1d(A,B))
    den=len(np.union1d(A,B))
    
    if den ==0:
        return 0
    else:
        return num/den

def gradientOrientation(grad,hemi,aparc):
    """Determine the orientation of the gradients, and also return whether valid for continued study or not"""
    grad=grad #nib.load(grad).agg_data()
    if hemi=='left':
        labels=nib.load(aparc).agg_data()
#       print('getting gradient orientation from left hemisphere')
    else:
        labels=nib.load(aparc).agg_data()
#       print('getting gradient orientation from right hemisphere')
    calc=np.where(labels==45)[0]
    ctr=np.where(labels==46)[0]
    if np.sum(grad[calc])<0 and np.sum(grad[ctr])<0:
#       print('Canonical Orientation DMN at apex')
        return grad,True
    elif np.sum(grad[calc])<0 and np.sum(grad[ctr])>0:
#       print(f'REMOVE {subj} FROM STUDY')
        return grad,False
    elif np.sum(grad[calc])>0 and np.sum(grad[ctr])<0:
#       print(f'REMOVE {subj} FROM STUDY')
        return grad,False
    else:
#       print('flipping gradient orientation for peak detection')
        return grad *-1,True


def prep_plotting(subj,kernel,sim='dice',pca=False):
    
    thr=[50,55,60,65,70,75,80,85,90,95]
    
    ctx_metric=[]
    zone_metricsL=[]
    zone_metricsR=[]
    
    if pca == False:
        gr=hcp_subj(subj,kernel)
        if gr.Lgradses1[1] == False or gr.Lgradses2[1] == False or gr.Rgradses1[1] ==False or gr.Rgradses2[1] ==False:
#             print(f'subject {gr.subj} Diffusion Mapping is not valid at smoothing kernel {kernel} ')
            return [gr.subj,kernel],[gr.subj,kernel],[gr.subj,kernel]
        else:
            if sim=='dice':
                for t in thr:
                    ctx_metric.append(gr.dice_Ses12(t))
                    zone_metricsL.append(gr.ZoneDice_Ses12(t)[0])
                    zone_metricsR.append(gr.ZoneDice_Ses12(t)[1])
            else: 
                for t in thr:
                    gr.Jaccard_Ses12(t)
                    ctx_metric.append(gr.Jaccard_Ses12(t))
                    zone_metricsL.append(gr.ZoneDice_Ses12(t)[0])
                    zone_metricsR.append(gr.ZoneDice_Ses12(t)[1])
                    
        
    else:
        gr=hcp_subj(subj,kernel,pca=True)
        if gr.Lgradses1[1] == False or gr.Lgradses2[1] == False or gr.Rgradses1[1] ==False or gr.Rgradses2[1] ==False:
#             print(f'subject {gr.subj} PCA is not valid at smoothing kernel {kernel} ')
            return [gr.subj,kernel],[gr.subj,kernel],[gr.subj,kernel]
        else:
            if sim=='dice':
                for t in thr:
                    ctx_metric.append(gr.dice_Ses12(t))
                    zone_metricsL.append(gr.ZoneDice_Ses12(t)[0])
                    zone_metricsR.append(gr.ZoneDice_Ses12(t)[1])
            else: 
                for t in thr:
                    gr.Jaccard_Ses12(t)
                    ctx_metric.append(gr.Jaccard_Ses12(t))
                    
                    zone_metricsL.append(gr.ZoneDice_Ses12(t)[0])
                    zone_metricsR.append(gr.ZoneDice_Ses12(t)[1])
                    
    return np.vstack(ctx_metric),np.vstack(zone_metricsL),np.vstack(zone_metricsR)


def prep_plotsXkernel(kernel,pca=False,corr=True):
    ### set up outputs
    Lhemi=[]
    Rhemi=[]
    LlatPar=[]
    LTmp=[]
    LmedPar=[]
    RlatPar=[]
    RTmp=[]
    RmedPar=[]
    
    nogo=[]
    
    ### loop through subjjects for specified kernel
    for subj in subjects:
        a,b,c=prep_plotting(subj,kernel,pca=pca)
        if len(a)>2:
            Lhemi.append(a[:,0])
            Rhemi.append(a[:,1])
        
            LlatPar.append(b[:,0])
            LTmp.append(b[:,1])
            LmedPar.append(b[:,2])
        
            RlatPar.append(c[:,0])
            RTmp.append(c[:,1])
            RmedPar.append(c[:,2])
        else:
            nogo.append(a[0])
    Lhemi=np.vstack(Lhemi)
    Rhemi=np.vstack(Rhemi)
    
    LlatPar=np.vstack(LlatPar)
    LTmp=np.vstack(LTmp)
    LmedPar=np.vstack(LmedPar)
    
    RlatPar=np.vstack(LlatPar)
    RTmp=np.vstack(LTmp)
    RmedPar=np.vstack(LmedPar)
    
    if corr == True:
        corr=(len(Lhemi)/len(subjects))
        return   (Lhemi*corr),(Rhemi*corr),(LlatPar*corr),(LTmp*corr),(LmedPar*corr),(RlatPar*corr),(RTmp*corr),(RmedPar*corr),nogo
    elif corr == False:
        return  Lhemi,Rhemi,LlatPar,LTmp,LmedPar,RlatPar,RTmp,RmedPar,nogo
    
def plot_itHue(k,rgn,legend=False,corr=True):
    """where k = kernel of 2,4,6,8,10 and rgn indexes the output of prep"""
    if corr==True:
        a=prep_plotsXkernel(k)
        b=prep_plotsXkernel(k,pca=True)
    elif corr==False:
        a=prep_plotsXkernel(k,corr=False)
        b=prep_plotsXkernel(k,pca=True,corr=False)

    a=pd.DataFrame.from_dict(dict(zip(thr,a[rgn].T)))
    a['Method']='Diffusion Mapping'
    b=pd.DataFrame.from_dict(dict(zip(thr,b[rgn].T)))
    b['Method']='PCA'

#     print(f'Dmaps has {len(a)} subjects')
#     print(f'PCA has {len(b)} subjects')
#     print(f'smoothing kernel is {k}')
    df=pd.concat([a,b])
    df=df.melt(id_vars=['Method'],value_vars=[50,55,60,65,70,75,80,85,90,95])
#     f, ax = plt.figure()
    ax=sn.boxplot(data=df,x='value',y='variable',hue='Method',orient='h')
    ax=sn.stripplot(data=df,x='value',y='variable',hue='Method',orient='h',size=3,dodge=True,palette=pal)
    
    plt.xlim([0,1])
    ax.set(ylabel = "")
    plt.xlabel(f"{k}mm Smoothing.\n Dmap with {len(a)}/20 \n PCA with {len(b)}/20 subjects", fontsize=12)
    ax.set(xlim=[0,1])
    
    if legend==False:
        ax.get_legend().remove()
    else: 
        ax.legend(bbox_to_anchor=(1.02, 0.55), loc='upper left', borderaxespad=0)
    
    plt.tight_layout()

def plot_srfs_dice(a,b,c,d,title,diceL,diceR):
    figure=plt.figure(figsize=(2,4), dpi=180)
    plt.subplot(2,2,1)
    plt.imshow(a)
    plt.title(diceL,fontsize=6)
    plt.axis('off')
    plt.subplot(2,2,2)
    
    plt.imshow(b)
    plt.axis('off')
    mlab.clf()
    plt.subplot(2,2,3)
    plt.imshow(c)
    plt.title(diceR,fontsize=6)
    plt.axis('off')
    plt.subplot(2,2,4)
    plt.imshow(d)
    plt.axis('off')
    
    figure.suptitle(title, fontsize=6)


    plt.subplots_adjust(left=0.1,
                        bottom=0.5, 
                        top=0.9, 
                        wspace=-0.1, 
                        hspace=0)
    mlab.close()
    return figure
#     plt.savefig(f'{file}.png',bbox_inches='tight',facecolor='w')
#     plt.close()


In [4]:
class hcp_subj:

    def __init__(self,subj,kernel,pca=None):
        
        self.subj=subj
        self.info=np.load(f'{subj}/{subj}.cifti.info.npy',allow_pickle=True).item()
        
        self.dims=self.info['lnverts']
        self.Lfill=self.info['lIDX']
        self.Rfill=self.info['rIDX']
        self.pca=pca
        
        self.Lsrf=f'../data20/{subj}/Structural/{subj}.L.midthickness.32k_fs_LR.surf.gii'
        self.Lcoords=nib.load(self.Lsrf).darrays[0].data
        self.Linflated='/Users/austin/Documents/ParisHorizontal/32k_fs_LR/S1200.L.inflated_MSMAll.32k_fs_LR.surf.gii'
        
        self.Rsrf=f'../data20/{subj}/Structural/{subj}.R.midthickness.32k_fs_LR.surf.gii'
        self.Rcoords=nib.load(self.Rsrf).darrays[0].data
        self.Laparc=f'../data20/{subj}/Structural/{subj}.L.aparc.a2009s.32k_fs_LR.label.gii'
        self.Rinflated='/Users/austin/Documents/ParisHorizontal/32k_fs_LR/S1200.R.inflated_MSMAll.32k_fs_LR.surf.gii'

        
        self.Raparc=f'../data20/{subj}/Structural/{subj}.R.aparc.a2009s.32k_fs_LR.label.gii'
        
    
        self.LZverts=get_zoneVerts(LWS)
        self.RZverts=get_zoneVerts(RWS)
    
    
        if self.pca is None:
            ########## session 1 
#             print('ussing diffusion maps')
            self.gradses1=np.load(f'{subj}/{subj}.mapalign.ses1.diffmap.s0{kernel}mm.npy')
        

        
            self.Lgradses1=self.gradses1[0][0:len(self.Lfill)]
            self.Lgradses1=recort(self.Lgradses1,self.Lfill,self.dims)
            self.Lgradses1=gradientOrientation(self.Lgradses1,'left',self.Laparc)
    
        
            self.Rgradses1=self.gradses1[0][len(self.Lfill):]
            self.Rgradses1=recort(self.Rgradses1,self.Rfill,self.dims)
            self.Rgradses1=gradientOrientation(self.Rgradses1,'right',self.Raparc)
        
            ######## session 2 
 
        
            self.gradses2=np.load(f'{subj}/{subj}.mapalign.ses2.s0{kernel}mm.diffmap.npy')
        
            self.Lgradses2=self.gradses2[0][0:len(self.Lfill)]
            self.Lgradses2=recort(self.Lgradses2,self.Lfill,self.dims)
            self.Lgradses2=gradientOrientation(self.Lgradses2,'left',self.Laparc)
    
        
            self.Rgradses2=self.gradses2[0][len(self.Lfill):]
            self.Rgradses2=recort(self.Rgradses2,self.Rfill,self.dims)
            self.Rgradses2=gradientOrientation(self.Rgradses2,'right',self.Raparc)
            
        else:
#             print('using PCA maps')
            ######### load PCA grads
            self.gradses1=np.load(f'{subj}/{subj}.pca.ses1.s0{kernel}mm.npy')
            self.Lgradses1=self.gradses1[0][0:len(self.Lfill)]
            self.Lgradses1=recort(self.Lgradses1,self.Lfill,self.dims)
            self.Lgradses1=gradientOrientation(self.Lgradses1,'left',self.Laparc)
    
        
            self.Rgradses1=self.gradses1[0][len(self.Lfill):]
            self.Rgradses1=recort(self.Rgradses1,self.Rfill,self.dims)
            self.Rgradses1=gradientOrientation(self.Rgradses1,'right',self.Raparc)
        
            self.gradses2=np.load(f'{subj}/{subj}.pca.ses2.s0{kernel}mm.npy')   
            self.Lgradses2=self.gradses2[0][0:len(self.Lfill)]
            self.Lgradses2=recort(self.Lgradses2,self.Lfill,self.dims)
            self.Lgradses2=gradientOrientation(self.Lgradses2,'left',self.Laparc)
    
        
            self.Rgradses2=self.gradses2[0][len(self.Lfill):]
            self.Rgradses2=recort(self.Rgradses2,self.Rfill,self.dims)
            self.Rgradses2=gradientOrientation(self.Rgradses2,'right',self.Raparc)
        
        
        
    
    def print_subj(self):
        print(self.subj)
    
    
    ### extract the vertices associated with each zone
    def zoning(self,Lgrad,Rgrad):
        Lg_zone=[]
        for key in self.LZverts:
            Lroi=self.LZverts[key]
            Lg_zone.append(Lgrad[0][Lroi])
        
        Rg_zone=[]

        for key in self.RZverts:
            Rroi=self.RZverts[key]
            Rg_zone.append(Rgrad[0][Rroi])
        return Lg_zone,Rg_zone

    
    def extract_topX(self,Left,Right,pct):
        """extract the top X percent instead of binning"""
        
        
        
        #######################################################
        #######################################################
        #######################################################
        # ADD CHECK HERE TO MAKE SURE IT IS A VALID GRADIENT WITH THE L[1] AND R[1]
        # THESE ARE SAVED AND HERE TO BE USED. DON'T WASTE THAT CHECK
        
        
        
        
        
        Left=Left[0]
        Right=Right[0]
        Lout=np.zeros(self.dims)
        Rout=np.zeros(self.dims)
        
        Lpct=np.percentile(Left[self.Lfill],pct)
        
        
        Lthr=np.where(Left[self.Lfill]>Lpct)[0]
        Linter=np.zeros(len(self.Lfill))
        Linter[Lthr]=1
        L=recort(Linter,self.Lfill,self.dims)
        L=np.where(L!=0)[0]
        
        #### do right 
        
                
        Rpct=np.percentile(Right[self.Rfill],pct)
        
        
        Rthr=np.where(Right[self.Rfill]>Rpct)[0]
        Rinter=np.zeros(len(self.Rfill))
        Rinter[Rthr]=1
        R=recort(Rinter,self.Rfill,self.dims)
        R=np.where(R!=0)[0]
        

        return L,R
    
    def ZoneDice_Ses12(self,pct):
        S1=self.extract_topX(self.Lgradses1,self.Rgradses1,pct)
        S2=self.extract_topX(self.Lgradses2,self.Rgradses2,pct)
        Ldx=[]
        for l in self.LZverts:
            roi=self.LZverts[l]
            S1ZnL=np.intersect1d(roi,S1[0])
            S2ZnL=np.intersect1d(roi,S2[0])
            diceLZn=dice_it(S1ZnL,S2ZnL)
            Ldx.append(diceLZn)
        Rdx=[]
        for r in self.RZverts:
            roi=self.RZverts[r]
            S1ZnR=np.intersect1d(roi,S1[1])
            S2ZnR=np.intersect1d(roi,S2[1])
            diceRZn=dice_it(S1ZnR,S2ZnR)
            Rdx.append(diceRZn)
        
        posteriorZones=[1,4,6]
        Ldx=[Ldx[i] for i in posteriorZones]
        Rdx=[Rdx[i] for i in posteriorZones]
        
        return np.asarray(Ldx),np.asarray(Rdx)
    
    
    
    def dice_Ses12(self,pct):
        S1=self.extract_topX(self.Lgradses1,self.Rgradses1,pct)
        S2=self.extract_topX(self.Lgradses2,self.Rgradses2,pct)
      
        diceL=dice_it(S1[0],S2[0])
        diceR=dice_it(S1[1],S2[1])
        
        
        return np.asarray([diceL,diceR])
    
    
    ############ implement the jaccard metric too
    def ZoneJaccard_Ses12(self,pct):
        S1=self.extract_topX(self.Lgradses1,self.Rgradses1,pct)
        S2=self.extract_topX(self.Lgradses2,self.Rgradses2,pct)
        Ldx=[]
        for l in self.LZverts:
            roi=self.LZverts[l]
            S1ZnL=np.intersect1d(roi,S1[0])
            S2ZnL=np.intersect1d(roi,S2[0])
            diceLZn=jaccard_it(S1ZnL,S2ZnL)
            Ldx.append(diceLZn)
        Rdx=[]
        for r in self.RZverts:
            roi=self.RZverts[r]
            S1ZnR=np.intersect1d(roi,S1[1])
            S2ZnR=np.intersect1d(roi,S2[1])
            diceRZn=jaccard_it(S1ZnR,S2ZnR)
            Rdx.append(diceRZn)
        
        posteriorZones=[1,4,6]
        Ldx=[Ldx[i] for i in posteriorZones]
        Rdx=[Rdx[i] for i in posteriorZones]
        
        return np.asarray(Ldx),np.asarray(Rdx)
    
    
    
    def Jaccard_Ses12(self,pct):
        S1=self.extract_topX(self.Lgradses1,self.Rgradses1,pct)
        S2=self.extract_topX(self.Lgradses2,self.Rgradses2,pct)
      
        diceL=jaccard_it(S1[0],S2[0])
        diceR=jaccard_it(S1[1],S2[1])
        
        
        return np.asarray([diceL,diceR])
    
    
    
  
        
    def plot_gradses1(self):
        a,b=oh_mayavi(self.Lsrf,self.Lgradses1[0],'CMRmap')
        mlab.clf()
        c,d=oh_mayavi(self.Rsrf,self.Rgradses1[0],'CMRmap')
        plot_srfs(a,b,d,c)
        
    def plot_gradses2(self):
        a,b=oh_mayavi(self.Lsrf,self.Lgradses2[0],'CMRmap')
        mlab.clf()
        c,d=oh_mayavi(self.Rsrf,self.Rgradses2[0],'CMRmap')
        plot_srfs(a,b,d,c)
        
    def plot_topX(self,Left,Right,pct):
        L=np.zeros(self.dims)
        R=np.zeros(self.dims)
        
        topX=self.extract_topX(Left,Right,pct)
        L[topX[0]]=1
        R[topX[1]]=1
        
        a,b=oh_mayavi(self.Lsrf,L,'terrain',False)
        mlab.clf()
        c,d=oh_mayavi(self.Rsrf,R,'terrain',False)
        plot_srfs(a,b,d,c)
        
        
    def plot_ThrIntersectCortex(self,pct):
        
        L=np.zeros(self.dims)
        R=np.zeros(self.dims)
        
        topXSes1=self.extract_topX(self.Lgradses1,self.Rgradses1,pct)
        topXSes2=self.extract_topX(self.Lgradses2,self.Rgradses2,pct)


        
        
        
        L1=topXSes1[0]
        L2=topXSes2[0]
        Linter=np.intersect1d(L1,L2)
        Lunion=np.union1d(L1,L2)
        
#         print(f'Left Dice is {dice_it(L1,L2)}')
#         print(f'Left Jaccard is {jaccard_it(L1,L2)}')
        
        L[Lunion]=5
        L[Linter]=10
        
        R1=topXSes1[1]
        R2=topXSes2[1]
        Rinter=np.intersect1d(R1,R2)
        Runion=np.union1d(R1,R2)
        
#         print(f'Right Dice is {dice_it(R1,R2)}')
#         print(f'Right Jaccard is {jaccard_it(R1,R2)}')
        
        R[Runion]=5
        R[Rinter]=10
        
        a,b=oh_mayavi(self.Linflated,L,'PuBuGn',False)
        mlab.clf()
        d,c=oh_mayavi(self.Rinflated,R,'PuBuGn',False)
        
#         a,b=oh_mayavi(self.Lsrf,L,'PuBuGn',False)
#         c,d=oh_mayavi(self.Rsrf,R,'PuBuGn',False)
        plot_srfs_dice(a,b,c,d,f'{self.subj} Threshold {pct}',f'Left Dice is {dice_it(L1,L2):.2f}',f'Right Dice is {dice_it(R1,R2):.2f}')
        
        

In [5]:
#### initialize the plotting backend for mayavi 
mlab.init_notebook('png',150,150)

Notebook initialized with png backend.


In [6]:
### get zone colormaps
from matplotlib.colors import ListedColormap
from matplotlib import cm
zoneColors=[(131,229,250),(131,229,250),(13,60,124),(13,60,124),
            (77,135,45),(77,135,45),
            (74,140,197),(74,140,197),
            (65,250,95),(65,250,95),
            (227,150,52),(227,150,52),
            (170,239,112),(170,239,112),
            (250,97,191),(250,97,191),
            (119,68,196),(119,68,196)]
colors = [(e[0] / 255.0, e[1] / 255.0, e[2] / 255.0) for e in zoneColors]
cmap = ListedColormap(colors)

In [ ]:
thr=[50,55,60,65,70,75,80,85,90,95]

In [ ]:
a,b,c=prep_plotting('100206',6,pca=True)

In [ ]:
pal = sn.color_palette(palette='Set2',n_colors=10)

In [ ]:
regions={}
regions['Left Hemisphere']=0
regions['Right Hemisphere']=1
regions['Left Lateral Parietal']=2
regions['Left Temporal']=3
regions['Left Medial Parietal']=4
regions['Right Lateral Parietal']=5
regions['Right Temporal']=6
regions['Right Medial Parietal']=7

kernels=[2, 4, 6, 8, 10]

In [ ]:
##### plot dice scores corrected for number of subjects from group 
for key in regions:
    f, ax = plt.subplots(ncols=5,nrows=1,figsize=(16,6))
    plt.subplot(151)
    plot_itHue(2,regions[key])
    plt.subplot(152)
    plot_itHue(4,regions[key])
    plt.subplot(153)
    plot_itHue(6,regions[key])
    plt.gca().set_title(key,fontsize = 20)
    plt.subplot(154)
    plot_itHue(8,regions[key])
    plt.subplot(155)
    plot_itHue(10,regions[key],legend=True)
    
    plt.savefig(f'../Dice_overlap/{key}_corr.png',facecolor='w')
    

In [ ]:
##### plot dice scores uncorrected for number of subjects left in group 
for key in regions:
    f, ax = plt.subplots(ncols=5,nrows=1,figsize=(16,6))
    plt.subplot(151)
    plot_itHue(2,regions[key],corr=False)
    plt.subplot(152)
    plot_itHue(4,regions[key],corr=False)
    plt.subplot(153)
    plot_itHue(6,regions[key],corr=False)
    plt.gca().set_title(key,fontsize = 20)
    plt.subplot(154)
    plot_itHue(8,regions[key],corr=False)
    plt.subplot(155)
    plot_itHue(10,regions[key],corr=False,legend=True)
    
    plt.savefig(f'../Dice_overlap/{key}_uncorr.png',facecolor='w')
    

In [ ]:
##### get rejects
pc_reject=[]
dm_reject=[]

dm_dirtySubj=[]
pc_dirtySubj=[]
kernels=[2,4,6,8,10]
for k in kernels:
    
    a=prep_plotsXkernel(k)
    dm_reject.append(len(subjects)-len(a[8]))
    dm_dirtySubj.append(a[8])
    
    b=prep_plotsXkernel(k,pca=True)
    pc_reject.append(len(subjects)-len(b[8]))
    pc_dirtySubj.append(b[8])
    
dm_reject=np.asarray(dm_reject)
pc_reject=np.asarray(pc_reject)

In [ ]:
# sn.set()
g=sn.lineplot(x=kernels,y=dm_reject,markers=True, dashes=True,marker='o',label='Dmap')
g=sn.lineplot(x=kernels,y=pc_reject,markers=True, dashes=True,marker='o',label='PCA')
g.set_xticks(kernels)
g.set_xlabel('Smoothing kernel')
g.set_ylabel('Subjects included in Dice')
g.set_title('Canonical first gradient')
plt.savefig('subjects_perkernel.png',facecolor='w')

In [ ]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [ ]:
mm6_dirt=dm_dirtySubj[2]

In [ ]:
dm_dirtySubj=set(flatten(dm_dirtySubj))
pc_dirtySubj=set(flatten(pc_dirtySubj))

In [ ]:
clean_6mm=list(set(subjects).difference(mm6_dirt))
clean_dm=list(set(subjects).difference(dm_dirtySubj))
clean_pc=list(set(subjects).difference(pc_dirtySubj))

In [ ]:
mlab.options.offscreen = True

In [ ]:
!mkdir -p indi6m_90

In [ ]:
for cl_subj in clean_6mm[0:8]:
    thr=90
    data=hcp_subj(cl_subj,6)
    data.plot_ThrIntersectCortex(thr)
    plt.savefig(f'indi6m_90/{data.subj}.dmap.thr0{thr}.s06mm.png',facecolor='w',pad_inches = 0.1,bbox_inches='tight')


In [7]:
thr=[50, 55, 60, 65, 70, 75, 80, 85, 90, 95]
kernels=[2,4,6,8,10]
!mkdir -p ../subject_dice

In [8]:
def plot_subjectThresholdsAndSmoothing_dmap(subj):
    cl_subj=subj
    kernels=[2,4,6,8,10]
    thr=[50,55,60,65,70,75,80,85,90,95]
    for k in kernels:
        !mkdir -p ./tmp
        data=hcp_subj(cl_subj,k)
        for i in thr:
            data.plot_ThrIntersectCortex(i)
            plt.savefig(f'tmp/{data.subj}.dmap.thr0{i}.s0{k}mm.png',facecolor='w',pad_inches = 0.1,bbox_inches='tight')
            plt.close()
            plt.cla()
            plt.clf()
            mlab.clf()
            mlab.close()
        !convert tmp/*png +append ../subject_dice/{data.subj}.dmap.s0{k}mm.png
        !rm tmp/*png

In [9]:
def plot_subjectThresholdsAndSmoothing_pca(subj):
    cl_subj=subj
    kernels=[2,4,6,8,10]
    thr=[50,55,60,65,70,75,80,85,90,95]
    for k in kernels:
        !mkdir -p ./tmp
        data=hcp_subj(cl_subj,k,pca=True)
        for i in thr:
            data.plot_ThrIntersectCortex(i)
            plt.savefig(f'tmp/{data.subj}.pca.thr0{i}.s0{k}mm.png',facecolor='w',pad_inches = 0.1,bbox_inches='tight')
            plt.close()
            plt.cla()
            plt.clf()
            mlab.clf()
            mlab.close()
        !convert tmp/*png +append ../subject_dice/{data.subj}.pca.s0{k}mm.png
        !rm tmp/*png

In [ ]:
plot_subjectThresholdsAndSmoothing_dmap('101309')

In [10]:
plot_subjectThresholdsAndSmoothing_pca('101309')


Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1


<Figure size 432x288 with 0 Axes>